In [2]:
import pyodbc
import pandas as pd
import numpy as np
#import os
import pickle
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import plotly.graph_objects as go
#from bqplot import pyplot as bplt
import seaborn as sns
import voila
from IPython.display import clear_output
import imageio as iio
from itables import init_notebook_mode, show

import warnings
warnings.filterwarnings("ignore")
#mpl.rc('font',family='Segoe UI')
#plt.style.use("seaborn-v0_8-white")
#os.chdir("C:\\Users\\gcubb\\OneDrive\\Family\\College")

In [3]:
def get_ts_data_multi(cdat,schnames,flds):
    #get all values of fld from cdat, which is a wide panel with year and UNITID fields
    maxyr = np.max(cdat.year) 
    idnm = cdat[cdat.year==maxyr][['UNITID','INSTNM']].set_index('UNITID')
    schids = idnm[idnm['INSTNM'].isin(schnames)]
    tsd = cdat[cdat.UNITID.isin(schids.index)][flds+['year','INSTNM']].set_index('year').sort_index()
    return tsd

def add_to_dat(origdata):
    ts_dat = origdata.rename(columns={'DVADM08':'admYieldMen','DVADM09':'admYieldWom','NPT430':'netPrcU75','NPT440':'netPrcU110','NPT450':'netPrcOver110'})
    ts_dat['ENRLFTM'] = ts_dat['ENRLFTM'].fillna(0)
    ts_dat['ENRLFTW'] = ts_dat['ENRLFTW'].fillna(0)
    ts_dat['percRoom'] = 100*ts_dat['ROOMCAP']/ts_dat['ADMSSN']
    ts_dat['percAdm'] = 100*ts_dat['ADMSSN']/ts_dat['APPLCN']
    ts_dat['percAdmMen'] = 100*ts_dat['ADMSSNM']/ts_dat['APPLCNM']
    ts_dat['percAdmWom'] = 100*ts_dat['ADMSSNW']/ts_dat['APPLCNW']
    ts_dat['approxUndergrad'] = 4*(ts_dat['ENRLFTM']+ts_dat['ENRLFTW'])
    ts_dat['approxPercWom'] = 100*ts_dat['ENRLFTW']/(ts_dat['ENRLFTM']+ts_dat['ENRLFTW'])
    ts_dat['approxPercMen'] = 100*ts_dat['ENRLFTM']/(ts_dat['ENRLFTM']+ts_dat['ENRLFTW'])
    ts_dat['citystate'] = ts_dat['CITY']+', '+ts_dat['STABBR']
    localemap = {11	:'City: Large'
    ,12	:'City: Midsize'
    ,13	:'City: Small'
    ,21	:'Suburb: Large'
    ,22	:'Suburb: Midsize'
    ,23	:'Suburb: Small'
    ,31	:'Town: Fringe'
    ,32	:'Town: Distant'
    ,33	:'Town: Remote'
    ,41	:'Rural: Fringe'
    ,42	:'Rural: Distant'
    ,43	:'Rural: Remote'
    ,-3	:'Not available'}
    ts_dat['localdesc'] = ts_dat['LOCALE'].map(localemap)
    return ts_dat

def pltmult(ax, df, varcho, xlab, ylab):
    ax = sns.lineplot(data=df, x=df.index, y=varcho[0], hue='INSTNM')
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)
    ax.legend()
    ax.grid()
    #set x axis gridlines to 1 year increments
    ax.set_xticks(df.index)
    return ax


In [4]:
fnm = 'ipeds_2014_2022'
with open(fnm+".pickle","rb") as f:
    cdat = pickle.load(f)

# College Data Comparison: Choose 4 schools
---
<ol>
<li>Enter a college name snippet in each text box</li>
<li>Select school from corresponding dropdown (if multiple names match snippet)</li>
<li>Hit button to run report</li>
</ol>

In [5]:
univsnippet1 = widgets.Text(value='William &',placeholder='Enter part of college name',description='Search #1:',disabled=False,continuous_update=False)
univsnippet2 = widgets.Text(value='Grinn',placeholder='Enter part of college name',description='Search #2:',disabled=False,continuous_update=False)
univsnippet3 = widgets.Text(value='Hamilton',placeholder='Enter part of college name',description='Search #3:',disabled=False,continuous_update=False)
univsnippet4 = widgets.Text(value='Smith',placeholder='Enter part of college name',description='Search #4:',disabled=False,continuous_update=False)
optionalTitle = widgets.Text(value='',placeholder='Enter chart title',description='Opt Title:',disabled=False,continuous_update=False)

schchoice1 = widgets.Dropdown(options=['William & Mary'],value='William & Mary',description='Choose #1:',disabled=False)
schchoice2 = widgets.Dropdown(options=['Grinnell College'],value='Grinnell College',description='Choose #2:',disabled=False)
schchoice3 = widgets.Dropdown(options=['Hamilton College'],value='Hamilton College',description='Choose #3:',disabled=False)
schchoice4 = widgets.Dropdown(options=['Smith College'],value='Smith College',description='Choose #4:',disabled=False)
out=widgets.Output()

def schlistm(snipval):
    schoollist=[]
    idnm = cdat[['UNITID','INSTNM']].set_index('UNITID')
    schid = idnm[idnm['INSTNM'].str.contains(snipval)]
    slist = schid.INSTNM.unique()
    schoollist = slist.copy()
    return schoollist

def create_data_multi(b=None):
    schnames = [schchoice1.value,schchoice2.value,schchoice3.value,schchoice4.value]
    flds = ['ADMSSN','APPLCN','ADMSSNM','APPLCNM','ADMSSNW','APPLCNW','SATVR25','SATVR75','SATMT25','ENRLFTM','ENRLFTW','SATMT75', 'DVADM08', 'DVADM09'
            ,'SATPCT','ACTPCT','NPT430','NPT440','NPT450','CALSYS','CITY','STABBR','OBEREG','LOCALE','ROOMCAP','RMBRDAMT']
    ts_dat_orig = get_ts_data_multi(cdat,schnames,flds)
    ts_dat = add_to_dat(ts_dat_orig)
    return ts_dat

# def create_data(b=None):
#     schname = schchoice.value
#     flds = ['ADMSSN','APPLCN','ADMSSNM','APPLCNM','ADMSSNW','APPLCNW','SATVR25','SATVR75','SATMT25','ENRLFTM','ENRLFTW'
#         ,'SATMT75', 'DVADM08', 'DVADM09', 'SATPCT','ACTPCT','NPT430','NPT440','NPT450'
#         ,'CALSYS','CITY','STABBR','OBEREG','LOCALE','ROOMCAP','RMBRDAMT']
#     ts_dat_orig = get_ts_data(cdat,schname,flds)
#     ts_dat = add_to_dat(ts_dat_orig)
#     return ts_dat

def create_table(ts_dat):
    table_flds = ['INSTNM','citystate','localdesc','APPLCN','ADMSSN','ROOMCAP','approxUndergrad','approxPercWom','approxPercMen','percRoom']
    tab = pd.DataFrame(ts_dat.loc[np.max(ts_dat.index),table_flds])
    format_mapping = {'APPLCN': '{:,.0f}'
                    ,'ADMSSN': '{:,.0f}'
                    ,'netPrcU75': '{:,.0f}'
                    ,'netPrcOver110': '{:,.0f}'
                    ,'ROOMCAP': '{:,.0f}'
                    ,'approxUndergrad': '{:,.0f}'
                    ,'approxPercWom': '{:.0f}%'
                    ,'approxPercMen': '{:.0f}%'
                    ,'percRoom': '{:.0f}%'
                    ,'percAdm': '{:.2f}%'
                    ,'percAdmMen': '{:.2f}%'
                    ,'percAdmWom': '{:.2f}%'
                    ,'SATPCT': '{:.2f}%'
                    ,'ACTPCT': '{:.2f}%'
                    }
    for key, value in format_mapping.items():
        if key in tab.columns:
            tab[key] = tab[key].apply(value.format)
    return tab


#def run_report_multi(tmult, schnamesarg):
def run_report_multi(tmult, opttitle):
    fig = plt.figure(figsize=(18,12))
    gs = fig.add_gridspec(2,15)
    if opttitle == '':
        fig.suptitle(f"4 Schools: Comparing Trends", fontsize=20, fontweight='bold')
    else:
        fig.suptitle(opttitle, fontsize=20, fontweight='bold')
    #fig.suptitle(f"4 Schools: Comparing Trends", fontsize=20, fontweight='bold')
    fig.subplots_adjust(hspace = 0.4)

    ax = fig.add_subplot(gs[0,0:7])
    varchoice = ['percAdm']
    ax.set_title("Admissions Rates: All", fontsize=12, fontweight='bold')
    ax = pltmult(ax, tmult, varchoice, "Year", "Admission Rate (%)")
    
    ax = fig.add_subplot(gs[0,8:15])
    varchoice = ['SATPCT']
    ax.set_title("Percent Submitting SAT", fontsize=12, fontweight='bold')
    ax = pltmult(ax, tmult.fillna(""), varchoice, "Year", "Submission Rate (%)")

    ax = fig.add_subplot(gs[1,0:7])
    varchoice = ['SATMT25']
    ax.set_title("Math Test Score Trends, 25th% (lowest)", fontsize=12, fontweight='bold')
    ax = pltmult(ax, tmult.fillna(""), varchoice, "Year", "Score")

    ax = fig.add_subplot(gs[1,8:15])
    varchoice = ['SATVR25']
    ax.set_title("Verbal  Test Score Trends, 25th% (lowest)", fontsize=12, fontweight='bold')
    ax = pltmult(ax, tmult.fillna(""), varchoice, "Year", "Score") 
    plt.close(fig)
    fig.savefig('collfig_4sch.png')
    fig.savefig('collfig_4sch.pdf')
    return fig

@univsnippet1.observe
def school_on_click(b):
    schchoice1.options=schlistm(univsnippet1.value)
@univsnippet2.observe
def school_on_click(b):
    schchoice2.options=schlistm(univsnippet2.value)
@univsnippet3.observe
def school_on_click(b):
    schchoice3.options=schlistm(univsnippet3.value)
@univsnippet4.observe
def school_on_click(b):
    schchoice4.options=schlistm(univsnippet4.value)

button = widgets.Button(description="Run Table/Graph")
output = widgets.Output()

searchboxes = widgets.VBox([univsnippet1,univsnippet2,univsnippet3,univsnippet4])
choiceboxes = widgets.VBox([schchoice1,schchoice2,schchoice3,schchoice4])
choicegrid = widgets.HBox([searchboxes,choiceboxes])
display(choicegrid, optionalTitle, button, output) 

#display(univsnippet, schchoice, button, output)

def fillwithmin(datain, colnames):
    for colname in colnames:
        fillval = datain[[colname]].fillna(9999).min()
        datain[colname] = datain[colname].fillna(fillval[0])
    return datain


def on_button_clicked(b):
    with output:
        clear_output()
        coldata = create_data_multi()
        #coldata = coldata.fillna('')
        coldata = fillwithmin(coldata, ['SATPCT','SATMT25','SATVR25'])
        tab = create_table(coldata)
        #schnames2 = [schchoice1.value,schchoice2.value,schchoice3.value]
        #ff2 = run_report_multi(coldata, schnames2)
        ff2 = run_report_multi(coldata,optionalTitle.value)
        tab = create_table(coldata)
        #print(tab)
        #qgridframe = qgrid.show_grid(tab, show_toolbar=True)
        #qgridframe
        init_notebook_mode(all_interactive=False)
        show(tab)
        img = mpimg.imread('collfig_4sch.png')
        fig, ax = plt.subplots(figsize=(18,12))
        im = ax.imshow(img)
        ax.axis('off')
        plt.show()

button.on_click(on_button_clicked)


Text(value='', continuous_update=False, description='Opt Title:', placeholder='Enter chart title')

Button(description='Run Table/Graph', style=ButtonStyle())

Output()

In [ ]:
# dtest = create_data_multi()
# dtest = fillwithmin(dtest, ['SATPCT','SATMT25','SATVR25'])
# table_flds = ['INSTNM','citystate','localdesc','APPLCN','ADMSSN','ROOMCAP','approxUndergrad','approxPercWom','approxPercMen','percRoom']
# tabtest = pd.DataFrame(dtest.loc[np.max(dtest.index),table_flds]).T
# #tabtest
# #qgridframe = qgrid.show_grid(tabtest, show_toolbar=True)
# #qgridframe
# init_notebook_mode(all_interactive=False)
# show(tabtest.T)